# Interactive web mapping
In this notebook I use the [ipyleaflet]() package to show how it can be used to add some interaction to a Juptyter notebook. One example is how to draw a bounding box on a map and use the geometry to filter the administrative boundary it belongs to. 

In [24]:
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles, Rectangle, DrawControl
from ipywidgets import Layout

m = Map(
    basemap=basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-04-08"),
    center=(41.890, 12.492),
    zoom=9
)
m.add(Marker(location=(41.890, 12.492)))
m.add(Marker(location=(42.0, 13.0)))
m.add(Marker(location=(41.7, 12.6)))
rectangle = Rectangle(bounds=((42.0, 13.0), (41.7, 12.6)), transform=True)
m.add(rectangle)
m

Map(center=[41.89, 12.492], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [2]:
rectangle.bounds

[(42.0, 13.0), (41.7, 12.6)]

In [23]:
watercolor = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)

m = Map(layers=(watercolor, ), center=(50, 354), zoom=5)

draw_control = DrawControl()
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 0.2
    }
}

feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}

def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""    
    feature_collection['features'].append(geo_json)

draw_control.on_draw(handle_draw)

m.add(draw_control)

m

Map(center=[50, 354], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [43]:
num_features = len(feature_collection['features'])
num_features

1

In [22]:
if (num_features > 0): 
    print(feature_collection['features'][num_features - 1]['geometry']['coordinates'])

[[[372.118022, 41.562854], [372.118022, 42.217127], [372.777445, 42.217127], [372.777445, 41.562854], [372.118022, 41.562854]]]


In [31]:
milan_gdf = countries = geopandas.read_file('data/vector/italy/osm/milan_20240310.geojson')

In [42]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl
import geopandas
import json

m = Map(center=(45.465, 9.188), zoom = 11, basemap= basemaps.OpenStreetMap.Mapnik)

geo_data = GeoData(geo_dataframe = milan_gdf,
                   style={'color': 'orange', 'fillColor': 'red', 'opacity':1.0, 'weight':1.0, 'dashArray':'2', 'fillOpacity':1.0},
                   hover_style={'fillColor': 'blue' , 'fillOpacity': 1.0},
                   name = 'Milan')

m.add(geo_data)
draw_control = DrawControl()
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 0.2
    }
}

feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}

def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""    
    feature_collection['features'].append(geo_json)

draw_control.on_draw(handle_draw)

m.add(draw_control)
m.add(LayersControl())

m


Map(center=[45.465, 9.188], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [44]:
num_features = len(feature_collection['features'])
num_features

1

In [45]:
if (num_features > 0): 
    print(feature_collection['features'][num_features - 1]['geometry']['coordinates'])

[[[9.174884, 45.497883], [9.174884, 45.523025], [9.223654, 45.523025], [9.223654, 45.497883], [9.174884, 45.497883]]]
